In [1]:
import requests
import json
import pandas as pd
from pathlib import Path
from MCForecastTools import MCSimulation

In [2]:
startdate = '2017-01-01'
enddate = '2017-01-31'
BTC = f"https://api.polygon.io/v2/aggs/ticker/X:BTCUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
ETH = f"https://api.polygon.io/v2/aggs/ticker/X:ETHUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
LTC = f"https://api.polygon.io/v2/aggs/ticker/X:LTCUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
DOGE = f"https://api.polygon.io/v2/aggs/ticker/X:DOGEUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
SOL = f"https://api.polygon.io/v2/aggs/ticker/X:SOLUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
DOT = f"https://api.polygon.io/v2/aggs/ticker/X:DOTUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
BNB = f"https://api.polygon.io/v2/aggs/ticker/X:BNBUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"
UNI = f"https://api.polygon.io/v2/aggs/ticker/X:UNIUSD/range/1/day/{startdate}/{enddate}?adjusted=true&sort=asc&limit=5000&apiKey=EuyLkPNR5tszoUvMnR5fZMjSdoBoNxIm"

In [3]:
url_list = [BTC, ETH, LTC, DOGE]

#url2_list = [SOL, DOT, BNB, UNI]


In [4]:
#function that accepts a variable string URL for pulling the json response file from polygon
def polygonurl(coinurl):
        response = requests.get(coinurl).json()
        return response

#function for turning the json file into the dataframe with required columns
def jsontodf(json):
        df = pd.json_normalize(json, record_path =['results'],
        meta =['ticker']
        )
        raw_data = pd.DataFrame(columns= ["t", "o", "h", "l", "c", "v", "n", "vw", "ticker"])
        df = pd.concat([raw_data, df])
        return df

#function for structuring dataframe so MCsimulation will accept as input
def structuredata(df):
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap', 'symbol']
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('timestamp')
        col_names = [(df["symbol"].iloc[0], x) for x in df.columns]
        df.drop('symbol')
        df.columns = pd.MultiIndex.from_tuples(col_names)
        return df


#function for saving an invidiual coin data as .csv, not currently being used
def savetocsv(df):
        filename = f'{df["symbol"].iloc[0]}'
        print(filename)
        filepath = Path(f'{filename} + ".csv"')
        df.to_csv(filepath)

#function to merge two dfs into MCsimulation format
def merger(df1, df2):
        df_merged = pd.merge(df1, df2, how = "inner", left_index=True, right_index=True)
        return df_merged


#run function that accepts a list of urls defined other variables needed for structuring the combined data
def run(list, currentdf, counter):
        if(list.index(counter) == 0):
                response = polygonurl(i)
                df = jsontodf(response)
                df = structuredata(df)
                #display(df.head())
                #print('hi')
                return df
        else:        
                response = polygonurl(i)
                df = jsontodf(response)
                df = structuredata(df)
                df = merger(currentdf, df)
                #display(df.head())
                #print('ho')
                return df



In [5]:
#cell for executing the data compiling functions

MCdf = pd.DataFrame()

#looping through the urls to build MCsimulation dataframe input
for i in url_list:
        MCdf = run(url_list, MCdf, i)

display(MCdf.head())

KeyError: 'results'

In [ ]:
# Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# Run 500 samples.
MC_equal_weight = MCSimulation(
    portfolio_data = MCdf,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
# Printing the first five rows of the simulation input data
MC_equal_weight.portfolio_data.head()

In [ ]:
MC_equal_weight.calc_cumulative_return()

In [ ]:
#MC_equal_weight_plot = MC_6040_weight.plot_simulation()
MC_equal_weight_dist = MC_equal_weight.plot_distribution()
MC_summary_stats = MC_equal_weight.summarize_cumulative_return()
print(MC_summary_stats)